# 7 - Métricas de evaluación


## Preparación del entorno
Fijamos semillas, comprobamos versiones y definimos utilidades.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer, load_digits, load_diabetes
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import (confusion_matrix, ConfusionMatrixDisplay,
    accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score,
    roc_curve, auc, precision_recall_curve, average_precision_score,
    matthews_corrcoef, log_loss, brier_score_loss, classification_report,
    mean_squared_error, mean_absolute_error, r2_score)
from sklearn.calibration import calibration_curve, CalibratedClassifierCV

import json, os, sys, platform, random

In [ ]:
# Reproducibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

## 1. Preparación para clasificación (binaria)

**Dataset**: `load_breast_cancer`

In [ ]:
data = load_breast_cancer(as_frame=True)
df_bin = data.frame.copy()
X_bin = df_bin.drop(columns=['target'])
y_bin = df_bin['target']

display(df_bin.head(3))
print(f"Dimensiones: {X_bin.shape}\n\nDistribución del {y_bin.value_counts(normalize=True)}")

Definimos un **holdout estratificado 60/20/20** (Train/Valid/Test)

In [ ]:
X_tmp, X_test, y_tmp, y_test = train_test_split(X_bin, y_bin, test_size=0.2, stratify=y_bin, random_state=SEED)
X_train, X_valid, y_train, y_valid = train_test_split(X_tmp, y_tmp, test_size=0.25, stratify=y_tmp, random_state=SEED)

print(f"Train:\t\t{len(X_train)} registros; tasa de prevalencia: {y_train.mean():.4f}")
print(f"Validation:\t{len(X_valid)} registros; tasa de prevalencia: {y_valid.mean():.4f}")
print(f"Test:\t\t{len(X_test)} registros; tasa de prevalencia: {y_test.mean():.4f}")

Se crea un **pipeline base** que se mantendrá a lo largo de las secciones para garantizar compatibilidad.


In [ ]:
pipe_bin = Pipeline([
    ('imp', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=SEED))
])
pipe_bin

Realizamos el entrenamiento mediante el pipeline con los datos de train, y obtenemos las **probabilidades predichas** para la clase positiva (target=1) en los conjuntos de **validación** y **test**.

In [ ]:
pipe_bin.fit(X_train, y_train)

proba_valid = pipe_bin.predict_proba(X_valid)[:,1]
proba_test  = pipe_bin.predict_proba(X_test)[:,1]

print(f"Media de 10 probabilidades predichas en validación: {round(proba_valid[:10].mean(), 3)}")

## 2. Matriz de confusión

Con umbral **τ = 0.5** calculamos la matriz de confusión en validación.

In [ ]:
# Establecer el umbral y aplicarlo a las predicciones
tau = 0.5
yhat_valid = (proba_valid >= tau).astype(int)

In [ ]:
# Generación de la matriz de confusión
cm = confusion_matrix(y_valid, yhat_valid, labels=[1,0])
disp = ConfusionMatrixDisplay(cm, display_labels=['Positivo','Negativo'])
fig, ax = plt.subplots(figsize=(4,4)); disp.plot(ax=ax, colorbar=False)
plt.show()

## 3. Métricas básicas (τ = 0.5)

* **Accuracy:** mide aciertos globales.
* **Precisión (PPV):** usado para minimizar FP.
* **Recall (TPR):** usado para minimizar FN.
* **Especificidad (TNR):** usado para minimizar FP también, aunque con otro enfoque.
* **FPR y FNR:** tasas de error por clase.

In [ ]:
acc = accuracy_score(y_valid, yhat_valid)
prec = precision_score(y_valid, yhat_valid, zero_division=0)
rec = recall_score(y_valid, yhat_valid)
tn, fp, fn, tp = confusion_matrix(y_valid, yhat_valid).ravel()
spec = tn / (tn + fp)
fpr = fp / (fp + tn)
fnr = fn / (fn + tp)

print("Métricas Básicas (τ = 0.5):")
print(f"  Accuracy: {acc:.4f}")
print(f"  Precision: {prec:.4f}")
print(f"  Recall (TPR): {rec:.4f}")
print(f"  Specificity (TNR): {spec:.4f}")
print(f"  FPR: {fpr:.4f}")
print(f"  FNR: {fnr:.4f}")

## 4. Métricas avanzadas

* **F1**: media armónica de Precisión y Recall.
* **Fβ**: pondera más el Recall cuando β > 1.
* **Balanced Accuracy**: media de TPR y TNR (útil con desbalance).
* **MCC**: Coeficiente de Correlación de Matthews (robusto con desbalance).


In [ ]:
f1 = f1_score(y_valid, yhat_valid)
bal_acc = balanced_accuracy_score(y_valid, yhat_valid)
mcc = matthews_corrcoef(y_valid, yhat_valid)

# Cálculo manual de F_beta
def fbeta_score_manual(y_true, y_pred, beta=2.0):
    p = precision_score(y_true, y_pred, zero_division=0); r = recall_score(y_true, y_pred)
    b2 = beta**2
    return (1 + b2) * p * r / (b2 * p + r + 1e-12)
f2 = fbeta_score_manual(y_valid, yhat_valid, beta=2.0)

print("Métricas Avanzadas:")
print(f"  F1: {f1:.4f}")
print(f"  F2: {f2:.4f}")
print(f"  Balanced Accuracy: {bal_acc:.4f}")
print(f"  MCC: {mcc:.4f}")

## 5. Multiclase: micro / macro / weighted

**Dataset:** `load_digits` tiene 10 clases.

Se realiza un holdout estratificado y se entrena mediante un pipeline sencillo.


In [ ]:
digits = load_digits(as_frame=True)
Xd, yd = digits.data, digits.target
Xd_tr, Xd_te, yd_tr, yd_te = train_test_split(Xd, yd, test_size=0.2, stratify=yd, random_state=SEED)

pipe_mc = Pipeline(
    [('imp', SimpleImputer(strategy='median')),
     ('scaler', StandardScaler()),
     ('clf', LogisticRegression(max_iter=2000, multi_class='auto', random_state=SEED))])

pipe_mc.fit(Xd_tr, yd_tr)

Calculamos F1-score **micro**, **macro** y **weighted**.

In [ ]:
yd_pred = pipe_mc.predict(Xd_te)

f1_micro = f1_score(yd_te, yd_pred, average='micro')
f1_macro = f1_score(yd_te, yd_pred, average='macro')
f1_weighted = f1_score(yd_te, yd_pred, average='weighted')

print(f"F1-score (micro): {f1_micro:.4f}")
print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")

print('\nReporte de clasificación automático:\n')
print(classification_report(yd_te, yd_pred))  # recorte visual

---
## 6. Curva ROC y AUC-ROC (binaria)
Calculamos la curva ROC en Validación y su AUC.


In [ ]:
fpr, tpr, thr = roc_curve(y_valid, proba_valid)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.5f}')
plt.plot([0,1],[0,1],'--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('Curva ROC - Validación')
plt.legend()
plt.show()

---
## 7. Curva Precision-Recall y AUC-PR (binaria)
Útil cuando la clase positiva es rara o cuando queremos controlar falsos positivos.


In [ ]:
prec_curve, rec_curve, thr_pr = precision_recall_curve(y_valid, proba_valid)
ap = average_precision_score(y_valid, proba_valid)
baseline = y_valid.mean()

plt.figure(figsize=(6, 6))
plt.plot(rec_curve, prec_curve, label=f'AP = {ap:.5f}')
plt.hlines(baseline, 0, 1, linestyles='--', label=f'Baseline (prevalencia) ={baseline:.2f}', color='orange')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall - Validación')
plt.legend(loc='center left')
plt.show()

## 8. Métricas basadas en probabilidad: Log-loss y Brier
Penalizan probabilidades mal calibradas (no solo el orden).


In [ ]:
ll_valid = log_loss(y_valid, proba_valid)
brier_valid = brier_score_loss(y_valid, proba_valid)

ll_test = log_loss(y_test, proba_test)
brier_test = brier_score_loss(y_test, proba_test)

print("Métricas basadas en probabilidad:")
print(f"  Log-loss:")
print(f"    Validación:\t{ll_valid:.4f}")
print(f"    Test:\t{ll_test:.4f}")
print(f"  Brier Score:")
print(f"    Validación:\t{brier_valid:.4f}")
print(f"    Test:\t{brier_test:.4f}")

## 9. Selección de umbral


Se realiza un barrido **τ ∈ [0,1]** para buscar aquel que maximice F1-score en Validación.

In [ ]:
taus = np.linspace(0,1,101)
f1s, precs, recs = [], [], []

for t in taus:
    yhat = (proba_valid >= t).astype(int)
    f1s.append(f1_score(y_valid, yhat))
    precs.append(precision_score(y_valid, yhat, zero_division=0))
    recs.append(recall_score(y_valid, yhat))

best_idx = int(np.nanargmax(f1s))
tau_star = float(taus[best_idx])
print(f"Mejor umbral (conjunto Validación): {tau_star}")

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(taus, f1s, label='F1')
plt.axvline(tau_star, linestyle='--', label=f'tau*={tau_star:.2f}', color='orange')
plt.xlabel('Umbral')
plt.ylabel('Métrica')
plt.title('Barrido de umbral en Validación')
plt.legend()
plt.show()

Se reportan las métricas en **Test** con ese umbral fijo.

In [ ]:
yhat_test = (proba_test >= tau_star).astype(int)

print(f"Métricas en Test con umbral óptimo ({tau_star:.2f}):")
print(f"  F1: {f1_score(y_test, yhat_test):.4f}")
print(f"  Precisión: {precision_score(y_test, yhat_test, zero_division=0):.4f}")
print(f"  Recall: {recall_score(y_test, yhat_test):.4f}")

## 10. Calibración de probabilidades


Se **calibra** usando solo el **Train** (con cross-validation interno)

In [ ]:
# Calibramos usando solo el train (con CV interna) y evaluamos en Validación
calib = CalibratedClassifierCV(estimator=pipe_bin, method='isotonic', cv=5)
calib.fit(X_train, y_train)
proba_valid_cal = calib.predict_proba(X_valid)[:,1]

**Métricas** y **comparación**:

En este caso el calibrado nmo ofrece mejores resultados porque el original ya estaba bien calibrado, y el tamaño reducido del dataset implica un mejor potencial del proceso de calibrado.

In [ ]:
res = {
    'AUC_valid_before': auc(*roc_curve(y_valid, proba_valid)[:2]),
    'AUC_valid_after': auc(*roc_curve(y_valid, proba_valid_cal)[:2]),
    'Brier_before': brier_score_loss(y_valid, proba_valid),
    'Brier_after': brier_score_loss(y_valid, proba_valid_cal),
    'LogLoss_before': log_loss(y_valid, proba_valid),
    'LogLoss_after': log_loss(y_valid, proba_valid_cal)
}

print("Comparación de Métricas (Validación):\n")
print(f"{'Métrica':<20} | {'Antes':<10} | {'Después':<10}")
print("-" * 45)
print(f"{'AUC':<20} | {res['AUC_valid_before']:.4f}     | {res['AUC_valid_after']:.4f}")
print(f"{'Brier Score':<20} | {res['Brier_before']:.4f}     | {res['Brier_after']:.4f}")
print(f"{'Log-loss':<20} | {res['LogLoss_before']:.4f}     | {res['LogLoss_after']:.4f}")

In [ ]:
prob_true_b, prob_pred_b = calibration_curve(y_valid, proba_valid, n_bins=10, strategy='uniform')
prob_true_c, prob_pred_c = calibration_curve(y_valid, proba_valid_cal, n_bins=10, strategy='uniform')

plt.figure(figsize=(12, 6))
plt.plot([0,1],[0,1],'--', label='Perfectamente calibrado')
plt.plot(prob_pred_b, prob_true_b, marker='o', label='Antes')
plt.plot(prob_pred_c, prob_true_c, marker='o', label='Después (calibrado)')
plt.xlabel('Prob. predicha')
plt.ylabel('Frac. positiva real')
plt.title('Diagrama de calibración (Validación)')
plt.legend()
plt.show()


## 11. Métricas de regresión (R², MSE/MAE, MAPE/SMAPE)

**Dataset:** `load_diabetes` porque tiene un target numérico continuo.

Se realiza un holdout sencillo y se entrena usando un nuevo pipeline para regresión.

In [ ]:
diab = load_diabetes(as_frame=True)
Xr, yr = diab.data, diab.target
Xr_tr, Xr_te, yr_tr, yr_te = train_test_split(Xr, yr, test_size=0.2, random_state=SEED)
pipe_reg = Pipeline([('imp', SimpleImputer()), ('scaler', StandardScaler()), ('model', Ridge())])
pipe_reg.fit(Xr_tr, yr_tr)

Cálculo de **métricas**:

In [ ]:
yr_pred = pipe_reg.predict(Xr_te)

mse = mean_squared_error(yr_te, yr_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(yr_te, yr_pred)
r2 = r2_score(yr_te, yr_pred)

# MAPE y SMAPE (con eps para evitar división por 0)
eps = 1e-8
mape = np.mean(np.abs((yr_te - yr_pred) / np.maximum(np.abs(yr_te), eps)))
smape = np.mean(2 * np.abs(yr_pred - yr_te) / (np.abs(yr_te) + np.abs(yr_pred) + eps))

print("Métricas de Regresión:")
print(f"  MSE: {mse:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")
print(f"  R2: {r2:.4f}")
print(f"  MAPE: {mape:.4f}")
print(f"  SMAPE: {smape:.4f}")

## Ejercicio

1. Utiliza nuevos **datasets** (y pipelines) para replicar el **cálculo de métricas** y gráficos presentes a lo largo de la práctica.
2. **Interpreta las métricas** obtenidas y razona sobre su comportamiento al realizar cambios en los pasos del flujo de inferencia.